In [ ]:
import pandas as pd
import numpy as np
import joblib
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import KNNImputer
from sklearn.pipeline import Pipeline

class APGARCleaner(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.sexo_moda_ = None
        self.imputer_ = None
        self.valid_indices_ = None

    def _normalizar_tipo_parto(self, df):
        """Normaliza todos los tipos de TIPO PARTO a versiones sin tildes"""
        if 'TIPO PARTO' in df.columns:
            df['TIPO PARTO'] = df['TIPO PARTO'].replace({
                'ESPONTANEO': 'ESPONTANEO',
                'ESPONTÃNEO': 'ESPONTANEO',
                'ESPONTÁNEO': 'ESPONTANEO',
                'CESAREA': 'CESAREA',
                'CESÃREA': 'CESAREA',
                'CESÁREA': 'CESAREA',
                'INSTRUMENTADO': 'INSTRUMENTADO'
            })
        return df

    def fit(self, X, y=None):
        # CLAVE: Resetear índices desde el inicio para evitar problemas
        df_clean = X.copy().reset_index(drop=True)
        original_length = len(df_clean)

        # Remover duplicados MANTENIENDO el seguimiento de índices
        df_clean = df_clean.drop_duplicates().reset_index(drop=True)

        relevant_columns = [
            'SEXO', 'PESO (Gramos)', 'TALLA (CentImetros)', 'TIEMPO DE GESTACION',
            'NUMERO CONSULTAS PRENATALES', 'TIPO PARTO', 'MULTIPLICIDAD EMBARAZO',
            'APGAR1', 'EDAD MADRE', 'NUMERO HIJOS NACIDOS VIVOS', 'NUMERO EMBARAZOS'
        ]
        existing_columns = [col for col in relevant_columns if col in df_clean.columns]
        df_clean = df_clean[existing_columns].copy()

        # Si existe APGAR2, remover filas donde sea null
        if 'APGAR2' in X.columns:
            df_clean = df_clean.dropna(subset=['APGAR2']).reset_index(drop=True)

        # Remover filas con valores críticos faltantes
        critical_cols = ['APGAR1', 'TIEMPO DE GESTACION']
        existing_critical = [col for col in critical_cols if col in df_clean.columns]
        if existing_critical:
            df_clean = df_clean.dropna(subset=existing_critical, how='any').reset_index(drop=True)

        # Guardar los índices válidos (ya reseteados)
        self.valid_indices_ = df_clean.index.copy()

        # NORMALIZAR TIPO PARTO
        df_clean = self._normalizar_tipo_parto(df_clean)

        # Procesar SEXO
        if 'SEXO' in df_clean.columns:
            df_clean['SEXO'] = df_clean['SEXO'].replace('INDETERMINADO', np.nan)
            if df_clean['SEXO'].isnull().any():
                self.sexo_moda_ = df_clean['SEXO'].mode()[0] if len(df_clean['SEXO'].mode()) > 0 else 'FEMENINO'

        # Configurar imputador
        imputation_cols = ['TIEMPO DE GESTACION', 'PESO (Gramos)', 'TALLA (CentImetros)']
        existing_imputation_cols = [col for col in imputation_cols if col in df_clean.columns]

        if existing_imputation_cols:
            self.imputer_ = KNNImputer(n_neighbors=5)
            if df_clean[existing_imputation_cols].isnull().any().any():
                self.imputer_.fit(df_clean[existing_imputation_cols])

        print(f"Fit completado: {original_length} → {len(df_clean)} filas")
        return self

    def transform(self, X):
        # CLAVE: Resetear índices desde el inicio
        df_clean = X.copy().reset_index(drop=True)
        original_length = len(df_clean)

        # IMPORTANTE: NO remover duplicados en transform para preservar longitud
        # Solo limpiar datos sin cambiar el número de filas
        # df_clean = df_clean.drop_duplicates().reset_index(drop=True)

        relevant_columns = [
            'SEXO', 'PESO (Gramos)', 'TALLA (CentImetros)', 'TIEMPO DE GESTACION',
            'NUMERO CONSULTAS PRENATALES', 'TIPO PARTO', 'MULTIPLICIDAD EMBARAZO',
            'APGAR1', 'EDAD MADRE', 'NUMERO HIJOS NACIDOS VIVOS', 'NUMERO EMBARAZOS'
        ]
        existing_columns = [col for col in relevant_columns if col in df_clean.columns]
        df_clean = df_clean[existing_columns].copy()

        # NORMALIZAR TIPO PARTO
        df_clean = self._normalizar_tipo_parto(df_clean)

        # Procesar SEXO
        if 'SEXO' in df_clean.columns:
            df_clean['SEXO'] = df_clean['SEXO'].replace('INDETERMINADO', np.nan)
            if hasattr(self, 'sexo_moda_') and self.sexo_moda_ is not None:
                df_clean['SEXO'] = df_clean['SEXO'].fillna(self.sexo_moda_)
            df_clean['SEXO'] = df_clean['SEXO'].fillna('FEMENINO')

        # Imputación con KNN
        imputation_cols = ['TIEMPO DE GESTACION', 'PESO (Gramos)', 'TALLA (CentImetros)']
        existing_imputation_cols = [col for col in imputation_cols if col in df_clean.columns]

        if existing_imputation_cols and hasattr(self, 'imputer_') and self.imputer_ is not None:
            if df_clean[existing_imputation_cols].isnull().any().any():
                try:
                    imputed_values = self.imputer_.transform(df_clean[existing_imputation_cols])
                    df_clean[existing_imputation_cols] = imputed_values
                except Exception as e:
                    print(f"Warning: KNN imputation failed: {e}")
                    # Fallback a mediana/defaults
                    pass

            # Fallback para valores aún faltantes
            for col in existing_imputation_cols:
                if df_clean[col].isnull().any():
                    defaults = {
                        'TIEMPO DE GESTACION': 38,
                        'PESO (Gramos)': 3200,
                        'TALLA (CentImetros)': 50
                    }
                    median_val = df_clean[col].median()
                    if pd.isna(median_val):
                        median_val = defaults.get(col, 0)
                    df_clean[col] = df_clean[col].fillna(median_val)

        # Valores por defecto para columnas numéricas
        numeric_defaults = {
            'NUMERO CONSULTAS PRENATALES': 6,
            'APGAR1': 8,
            'EDAD MADRE': 25,
            'NUMERO HIJOS NACIDOS VIVOS': 1,
            'NUMERO EMBARAZOS': 1
        }

        for col, default_val in numeric_defaults.items():
            if col in df_clean.columns and df_clean[col].isnull().any():
                df_clean[col] = df_clean[col].fillna(default_val)

        # Valores por defecto para columnas categóricas
        categorical_defaults = {
            'TIPO PARTO': 'ESPONTANEO',
            'MULTIPLICIDAD EMBARAZO': 'SIMPLE'
        }

        for col, default_val in categorical_defaults.items():
            if col in df_clean.columns and df_clean[col].isnull().any():
                df_clean[col] = df_clean[col].fillna(default_val)

        # Convertir a categóricas
        categorical_cols = ['SEXO', 'TIPO PARTO', 'MULTIPLICIDAD EMBARAZO']
        for col in categorical_cols:
            if col in df_clean.columns:
                df_clean[col] = df_clean[col].astype('category')

        # One-hot encoding
        if 'TIPO PARTO' in df_clean.columns:
            df_clean = pd.get_dummies(df_clean, columns=['TIPO PARTO'], drop_first=False)

        if 'MULTIPLICIDAD EMBARAZO' in df_clean.columns:
            df_clean = pd.get_dummies(df_clean, columns=['MULTIPLICIDAD EMBARAZO'], drop_first=False)

        if 'SEXO' in df_clean.columns:
            df_clean = pd.get_dummies(df_clean, columns=['SEXO'], drop_first=True)

        # Crear índice de masa neonatal
        if 'PESO (Gramos)' in df_clean.columns and 'TALLA (CentImetros)' in df_clean.columns:
            # Evitar división por cero
            df_clean['TALLA (CentImetros)'] = df_clean['TALLA (CentImetros)'].replace(0, np.nan)
            df_clean['TALLA (CentImetros)'] = df_clean['TALLA (CentImetros)'].fillna(50)

            df_clean['INDICE_MASA_NEONATAL'] = df_clean['PESO (Gramos)'] / (df_clean['TALLA (CentImetros)'] ** 2)
            df_clean['INDICE_MASA_NEONATAL'] = df_clean['INDICE_MASA_NEONATAL'].replace([np.inf, -np.inf], np.nan)
            df_clean['INDICE_MASA_NEONATAL'] = df_clean['INDICE_MASA_NEONATAL'].fillna(1.28)

        # Eliminar columnas innecesarias
        cols_to_drop = [
            'PESO (Gramos)', 'TALLA (CentImetros)', 'EDAD MADRE',
            'NUMERO HIJOS NACIDOS VIVOS', 'NUMERO EMBARAZOS'
        ]

        dummy_cols_to_drop = [
            'TIPO PARTO_INSTRUMENTADO', 'MULTIPLICIDAD EMBARAZO_DOBLE',
            'MULTIPLICIDAD EMBARAZO_SIMPLE', 'MULTIPLICIDAD EMBARAZO_TRIPLE',
            'SEXO_MASCULINO'
        ]

        all_cols_to_drop = cols_to_drop + dummy_cols_to_drop
        existing_cols_to_drop = [col for col in all_cols_to_drop if col in df_clean.columns]

        if existing_cols_to_drop:
            df_clean = df_clean.drop(columns=existing_cols_to_drop)

        # Llenar cualquier valor faltante restante
        df_clean = df_clean.fillna(0)

        # VERIFICACIÓN FINAL - debe mantener el mismo número de filas
        final_length = len(df_clean)
        if final_length != original_length:
            print(f"ADVERTENCIA: Cambio en número de filas: {original_length} → {final_length}")
        else:
            print(f"Filas preservadas: {original_length}")

        # Asegurar que el DataFrame tenga índices consecutivos
        df_clean = df_clean.reset_index(drop=True)

        return df_clean

    def fit_transform(self, X, y=None):
        return self.fit(X, y).transform(X)

# Función para crear pipeline seguro
def crear_pipeline_seguro():
    """Crea un pipeline que maneja correctamente los índices"""
    print("Cargando modelo...")
    try:
        modelo_mlp = joblib.load('modelo_mlp_final.pkl')
        print("Modelo cargado exitosamente")
    except Exception as e:
        print(f"Error cargando modelo: {e}")
        return None

    pipeline = Pipeline([
        ('limpieza', APGARCleaner()),
        ('modelo', modelo_mlp)
    ])

    return pipeline

# Función para probar el pipeline
def probar_pipeline(pipeline, df_test):
    """Prueba el pipeline con datos de ejemplo"""
    try:
        print(f"Datos de entrada: {len(df_test)} filas")
        predicciones = pipeline.predict(df_test)
        print(f"Predicciones generadas: {len(predicciones)}")
        print("Pipeline funcionando correctamente")
        return predicciones
    except Exception as e:
        print(f"Error en pipeline: {e}")
        return None

if __name__ == "__main__":
    # Crear y guardar el pipeline corregido
    pipeline = crear_pipeline_seguro()

    if pipeline is not None:
        print("Guardando pipeline corregido...")
        joblib.dump(pipeline, 'pipeline_apgar_streamlit.pkl')
        print("✅ Pipeline corregido y guardado como 'pipeline_apgar_streamlit.pkl'")
        print("✅ Manejo de índices mejorado para Streamlit")
        print("✅ Soporte completo para tildes: CESÁREA/CESÃREA → CESAREA")
        print("✅ Soporte completo para tildes: ESPONTÁNEO/ESPONTÃNEO → ESPONTANEO")
    else:
        print("❌ No se pudo crear el pipeline")

Cargando modelo...
✅ Modelo cargado exitosamente
Guardando pipeline corregido...
✅ Pipeline corregido y guardado como 'pipeline_apgar_streamlit.pkl'
✅ Manejo de índices mejorado para Streamlit
✅ Soporte completo para tildes: CESÁREA/CESÃREA → CESAREA
✅ Soporte completo para tildes: ESPONTÁNEO/ESPONTÃNEO → ESPONTANEO
